In [131]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd


In [132]:
url = 'http://sofifa.com/players?offset=0'

In [133]:
fifa_stats = ['Crossing', 'Finishing', 'Heading Accuracy','Short Passing','Volleys', 
              'Dribbling', 'Curve','Free Kick Accuracy', 'Long Passing', 'Ball Control',
              'Acceleration', 'Sprint Speed', 'Agility', 'Reactions','Balance', 
              'Shot Power', 'Jumping', 'Stamina', 'Strength','Long Shots',
              'Aggression', 'Interceptions', 'Positioning','Vision', 'Penalties', 'Composure', 
              'Marking', 'Standing Tackle','Sliding Tackle',
              'GK Diving', 'GK Handling', 'GK Kicking','GK Positioning', 'GK Reflexes']

In [134]:
def soup_maker(url):
    r = requests.get(url)
    markup = r.content
    soup = bs(markup, 'lxml')
    return soup

In [171]:
soup = soup_maker(url)
table = soup.find('table',{'class':'table'})
tbody = table.find('tbody')
all_tr = tbody.findAll('tr')
all_players = []
for player in all_tr:
    player_details = {}
    player_details["shortname"] = player.select('div > a')[1].get_text(strip=True)
    soup2 = soup_maker("http://sofifa.com" + player.select('div > a')[1].get('href'))
    center = soup2.find('div',{'class':'center'})
    article = center.find('article')
    all_stats_soup = article.select('div')[25]
    list_of_stats = all_stats_soup.findAll('span',{'class':'label'})
    if len(list_of_stats) == 0:
        all_stats_soup = article.select('div')[26]
        list_of_stats = all_stats_soup.findAll('span',{'class':'label'})
    for i in range(20):
        player_details[fifa_stats[i]] = list_of_stats[i].find(text=True)
    all_stats_soup = article.select('div')[34]
    list_of_stats = all_stats_soup.findAll('span',{'class':'label'})
    if len(list_of_stats) == 5:
        all_stats_soup = article.select('div')[35]
        list_of_stats = all_stats_soup.findAll('span',{'class':'label'})
    for i in range(14):
        player_details[fifa_stats[i]] = list_of_stats[i].find(text=True)
    all_players.append(player_details)

In [174]:
df = pd.DataFrame(all_players)

In [175]:
df.describe()

,Acceleration,Agility,Balance,Ball Control,Crossing,Curve,Dribbling,Finishing,Free Kick Accuracy,Heading Accuracy,...,Long Passing,Long Shots,Reactions,Short Passing,Shot Power,Sprint Speed,Stamina,Strength,Volleys,shortname
count,61,61,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
unique,21,21,40,21,39,40,23,36,42,36,...,40,39,21,35,33,25,29,35,32,61
top,11,5,66,15,87,90,86,88,89,12,...,84,79,6,77,86,7,75,78,82,M. Neuer
freq,8,8,5,8,3,6,9,6,6,4,...,3,4,7,4,5,7,8,5,5,1


In [176]:
df.head(20)

,Acceleration,Agility,Balance,Ball Control,Crossing,Curve,Dribbling,Finishing,Free Kick Accuracy,Heading Accuracy,...,Long Passing,Long Shots,Reactions,Short Passing,Shot Power,Sprint Speed,Stamina,Strength,Volleys,shortname
0,11,14,95,6,48,33,96,22,28,94,...,26,94,8,94,85,15,72,66,75,L. Messi
1,11,14,70,7,63,28,95,29,31,95,...,23,93,11,82,95,15,88,79,85,Cristiano Ronaldo
2,9,15,84,9,56,27,94,36,24,89,...,33,82,11,88,80,15,81,49,81,Neymar Jr
3,85,89,43,90,38,25,70,30,21,12,...,13,12,94,68,31,85,43,64,40,De Gea
4,13,10,77,15,76,68,90,61,58,87,...,51,90,13,94,91,5,89,75,79,K. De Bruyne
5,12,8,94,11,54,34,91,41,27,87,...,22,80,8,89,82,6,83,66,86,E. Hazard
6,9,14,94,13,62,68,92,83,76,79,...,73,82,9,92,79,7,89,58,82,L. Modrić
7,25,33,83,27,87,62,85,41,45,92,...,38,85,37,84,86,31,90,83,85,L. Suárez
8,10,14,73,8,78,56,91,35,36,93,...,38,86,11,81,88,11,89,84,90,H. Kane
9,92,90,49,86,34,27,70,19,12,11,...,18,12,89,65,22,78,41,78,11,J. Oblak
